In [9]:
#create the sample with Sobol and all the IDFs files to simulation
import sys
import os
from SALib.sample import saltelli
from SALib.analyze import sobol
import numpy as np
from eppy import modeleditor
from eppy.modeleditor import IDF
import sqlite3
import pandas as pd

#folder with all database. This direcotry will have all IDF to run simulations
benchmark = "C:/sim/dissertacao metamodelo/"

#folder with idf parts
parts = 'D:/Tiago backup/Labeee/metamodelo hospitais SG/parts idf/'
#folder where will be created the data base
amostras = 'D:/Tiago backup/Labeee/metamodelo hospitais SG/amostras/'
#folder to word with idfs
work = 'D:/Tiago backup/Labeee/metamodelo hospitais SG'
#idd to select the E+ version
iddfile = 'C:/EnergyPlusV9-2-0/Energy+.idd'
#base idf file that will be changed to create the database
base_file = 'D:/Tiago backup/Labeee/metamodelo hospitais SG/HOSPITAL_BASE_IDL.idf'
#epw file to work with eppy will not be used in the simulations
epwfile = 'C:/Users/Tiago/Desktop/Work/BRA_SC_Florianopolis-Luz.AP.838990_TRY.1963.epw'
IDF.setiddname(iddfile)

#Set the base IDF
base_idf = IDF(base_file)

In [10]:
# list of idfs with differents constructions and other variables
list_tipo_cob = ['cob_0.33', 'cob_0.95', 'cob_2.0','cob_1.5']
list_tipo_par = ['par_0.11', 'par_0.63','par_1.8','par_2.6', 'par_4.4']
list_par_int = ['par1_ar','par2_la']
list_absortancia_cob = [0.2, 0.5, 0.8] 
list_absortancia_par = [0.2, 0.5, 0.8]
list_FS = ['FS_0.29','FS_0.37']
list_DPI = [8.7, 10.5, 15] #W/m2

#auxiliar lists to work with idf parts
paredes = []
paredes_int = []
coberturas = []
absortancias_par = []
absortancias_cob = []
Fator_Solar = []
Densidade_ilumin = []
nome_idf = []
nome_output = []

In [13]:
#Criação da amostra
# Getting the sample of the list of idfs        
problem = {
    'num_vars': 7,
    'names':['TipoCobertura', 'TipoParede', 'AbsortanciaCobertura', 
    'AbsortanciaParede', 'FatorSolar','DPI'],
    'bounds': [[0, len(list_tipo_cob)], 
    [0, len(list_tipo_par)], 
    [0, len(list_absortancia_cob)],
    [0, len(list_absortancia_par)],
    [0, len(list_FS)],
    [0,len(list_DPI)],
    [0,len(list_par_int)]],
    }
#problem from saltelli
param_values = saltelli.sample(problem,8)
#print(param_values)

idf_amostra = []
#convert the param_Values to int since we only have int variables
for i in range(len(param_values)):
    amostra = [int(valor) for valor in param_values[i]]
    idf_amostra.append(amostra)

#print "amostra" to verify the database
#print("IDF Amostra:", idf_amostra)
#print (len(idf_amostra))
    
#Pega os valores de cada lista
for i in range(len(idf_amostra)):
    parametros_indices = idf_amostra[i]
    tipo_cob = list_tipo_cob[parametros_indices[0]]
    tipo_par = list_tipo_par[parametros_indices[1]]
    absortancia_cob = list_absortancia_cob[parametros_indices[2]]
    absortancia_par = list_absortancia_par[parametros_indices[3]]
    FS = list_FS[parametros_indices[4]]
    DPI = list_DPI[parametros_indices[5]]
    par_int = list_par_int[parametros_indices[6]]
    
    #Create lists with information about the Sample
    paredes.append(tipo_par)
    coberturas.append(tipo_cob)
    absortancias_cob.append(absortancia_cob)
    absortancias_par.append(absortancia_par)
    Fator_Solar.append(FS)
    Densidade_ilumin.append(DPI)
    paredes_int.append(par_int)

    #Name de IDF for simulation
    if tipo_par == 'par_2.6' or tipo_par == 'par_4.4':
        name_output = ( tipo_cob + '_' + tipo_par + '_' + 'par_2.6' + '_' + str(absortancia_cob) + '_' + str(absortancia_par) + '_' + (FS) + '_' + str(DPI) + '_'  + '.csv')
        name_idf = ( tipo_cob + '_' + tipo_par + '_'  + 'par_2.6' + '_' + str(absortancia_cob) + '_' + str(absortancia_par) + '_' + (FS) + '_' + str(DPI) + '_'  + '.idf')
    else:
        name_output = ( tipo_cob + '_' + tipo_par + '_' + par_int + '_' + str(absortancia_cob) + '_' + str(absortancia_par) + '_' + (FS) + '_' + str(DPI) + '_'  + '.csv')
        name_idf = ( tipo_cob + '_' + tipo_par + '_'  + par_int + '_' + str(absortancia_cob) + '_' + str(absortancia_par) + '_' + (FS) + '_' + str(DPI) + '_'  + '.idf')
    #Create a list with IDF names
    nome_output.append(name_output)
    nome_idf.append(name_idf)

    os.chdir(work)

    #change directory to get idf parts for each variable   
    os.chdir(parts)
    #Reads the IDF part for each input
    cob_file = (tipo_cob + ".idf")
    par_file = (tipo_par + ".idf")
    par_int_file = (par_int + ".idf")
    fs_file = (FS + ".idf")
    
    #Read the file as IDF with Eppy library
    base_idf = IDF(base_file)
    cob_idf = IDF(cob_file)
    par_idf = IDF(par_file)
    fs_idf = IDF(fs_file)
    par_int_idf = IDF(par_int_file)
    os.chdir(work)


    #Criaçaõ dos IDFS da amostra

    #Updates materials absorptances of cob and par and inserts into base IDF
    base_materials = base_idf.idfobjects["MATERIAL"]

    cob_materials = cob_idf.idfobjects["MATERIAL"]
    for material in cob_materials:
        material.Solar_Absorptance = absortancia_cob
        material.Visible_Absorptance = absortancia_cob
#        print(material)
        base_materials.append(material)

    par_materials = par_idf.idfobjects["MATERIAL"]
    for material in par_materials:
        material.Solar_Absorptance = absortancia_par
        material.Visible_Absorptance = absortancia_par
#        print(material)
        base_materials.append(material)

    par_int_materials = par_int_idf.idfobjects["MATERIAL"]
    for material in par_int_materials:
        base_materials.append(material)

    fs_materials = fs_idf.idfobjects["WINDOWMATERIAL:GLAZING"]
    for material in fs_materials:
        base_materials.append(material)


#Inserts cob (roof) and par (walls) constructions into base IDF
    base_constructions = base_idf.idfobjects["CONSTRUCTION"]

    cob_construction = cob_idf.idfobjects["CONSTRUCTION"]
    for construction in cob_construction:
        base_constructions.append(construction)

    par_construction = par_idf.idfobjects["CONSTRUCTION"]
    for construction in par_construction:
        base_constructions.append(construction)

    fs_construction = fs_idf.idfobjects["CONSTRUCTION"]
    for construction in fs_construction:
        base_constructions.append(construction)

#Insert internal walls in IDF
    internal_walls_construction = par_int_idf.idfobjects["CONSTRUCTION"]
    if tipo_par == 'par_0.11'or tipo_par == 'par_0.63' or tipo_par =='par_1.8':
        for construction in internal_walls_construction:
            base_constructions.append(construction)
     
    #Updates DPI
    base_DPI = base_idf.idfobjects["LIGHTS"]
    for light in base_DPI:
        light.Watts_per_Zone_Floor_Area = DPI

    os.chdir(amostras)
    base_idf.saveas(name_idf)

In [ ]:
#Create a CSV file with the CSV name for simulation output and every input data of the sample
#Create a dictionary with variables names and characteristics
os.chdir(work)
dict_variaveis = { 'CSV': nome_output, 'Transmitância Cobertura': coberturas, 'Transmitância Parede': paredes, 'absortancia cob': absortancias_cob, 'absortancia parede': absortancias_par,
'Fator Solar': Fator_Solar, 'Densidade de Iluminacao': Densidade_ilumin}
#print(dict_variaveis)

#Create a CSV file containing all the cases for simulation
csv_variaveis = pd.DataFrame(dict_variaveis)
csv_variaveis.to_csv('VARIAVEIS_HOSPITAL'+ '.csv')


In [ ]:
#Não precisa olhar agora. Só para a fase de tratamento de dados
#Identify the file in folder with the same name of the CSV to get the energy consumption
csv_casos = pd.read_csv('D:/Tiago backup/Mestrado/metamodelo hospitais SG/VARIAVEIS_HOSPITAL.csv')
path = 'D:/Tiago backup/Mestrado/metamodelo hospitais SG/teste_resultados'
dfs = []
csvs = []
for i in range(len(path)):
    nome= csv_casos['CSV'][i]  
    for file in os.listdir(path):
        if file.endswith('.csv'):
            if nome == file:
                #print(file)
                csv = file   
                csvs.append(file)                         
                #calcula o consumo por uso
                df = pd.read_csv(csv)
                
                series = [df.filter(regex = r'(cooling )'), #CORRIGIR O FILTRO 
                df.filter(regex = r'(heating)'), #CORRIGIR O FILTRO 
                df.filter(regex = r'(iluminacao)'), #CORRIGIR O FILTRO 
                df.filter(regex = r'(equipamentos)'), #CORRIGIR O FILTRO 
                df.filter(regex = r'(Heating)')] #CORRIGIR O FILTRO             
            
                resultados = []
                
                for serie in series:
                        serie1 = serie.sum()*1/1 #CORRIGIR CONVERSAO DEPOIS 
                        serie2 = serie1.sum()
                        resultados.append(serie2)
                df1 = pd.DataFrame([resultados]).T
                dfs.append(df1)

casos = csvs       
results_uso = pd.concat(dfs, axis = 1)
results_total = pd.Series(results_uso.sum())
resultados =results_total.reset_index()
resultados['Casos'] = casos
resultados   